# Loading Model


Install required Libraries

In [ ]:
!pip install -q transformers einops
!pip install git+https://github.com/huggingface/accelerate
!pip install transformers[torch]
!pip install git+https://github.com/huggingface/peft.git
!pip install datasets bitsandbytes wandb trl
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 68.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 81.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.1 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-mmb55x_x
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-mmb55x_x
  Resolved https://github.com/huggingface/accelerate to commit c872c3086fb57a770d8dc656a14157a70e534139
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.23.0.dev0-py3-none-any.whl size=256461 sha256=dcd1b002aef1fa52c56cf0191e1faaa74e58c2ad83cb505bd04aab4

Load the libraries and Load the Tokenizer

In [ ]:
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer


model_name = "tiiuae/falcon-rw-1b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant = True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-rw-1b:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-rw-1b:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Build the Model Pipeline using Hugging Face Transformers Pipeline

In [ ]:
from peft import LoraConfig

lora_alpha = 32
lora_dropout = 0.05
lora_r = 16

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ]
)


# Dataset


In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelWithLMHead

# Load the dataset
dataset_name = "pubmed_qa"
dataset = load_dataset(dataset_name, "pqa_artificial")
tokenizer=AutoTokenizer.from_pretrained('T5-small')
model=AutoModelWithLMHead.from_pretrained('T5-small', return_dict=True)

# Split the dataset
dataset_reformat = dataset['train'].train_test_split(test_size=9000, train_size=21000)

# Define a function to combine context
def combine_context(row):
    row['context'] = ''.join(row['context']['contexts'])
    words = torch.split(row['context'], " ")
    word_count = len(words)
    if word_count > 512:
      inputs=tokenizer.encode("sumarize: " +row['context'],return_tensors='pt', max_length=512, truncation=True)
      output = model.generate(inputs, min_length=80, max_length=512)
      summary=tokenizer.decode(output[0])
      row['context'] = summary
    return row

# Define a function to get text
def get_text(row):
    row['text'] = "### Context: " + row['context'] + "### Question: " + row['question'] + "### Answer: " + row['long_answer']
    return row



# Process the dataset
dataset_train = dataset_reformat['train']
dataset_train = dataset_train.map(combine_context)
dataset_train = dataset_train.map(get_text, remove_columns=dataset_reformat["train"].column_names)




Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/211269 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1479: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


Map:   0%|          | 0/21000 [00:00<?, ? examples/s]

Map:   0%|          | 0/21000 [00:00<?, ? examples/s]

In [ ]:
print(dataset_train)

Dataset({
    features: ['text'],
    num_rows: 21000
})


# Loading the trainer

In [ ]:
from transformers import TrainingArguments



training_arguments = TrainingArguments(

    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs = 1,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=5,
    max_steps = 500,
    optim = "paged_adamw_32bit",
    output_dir = "./results",
    lr_scheduler_type = "constant",
    warmup_ratio = 0.03,
    max_grad_norm = 0.3,
    save_steps = 10,
    group_by_length=True,
    report_to=None
)



Pass everything to Trainer

In [ ]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_train,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/21000 [00:00<?, ? examples/s]

pre-process the model by upcasting the layer norms in float 32 for more stable training

In [ ]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)



# Train The Model

In [ ]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
5,1.908400
10,1.840000
15,1.889500
20,1.831400
25,1.868400
30,1.944300
35,1.925000
40,1.997700
45,1.945400
50,2.119800


TrainOutput(global_step=500, training_loss=1.8966534461975098, metrics={'train_runtime': 3759.3331, 'train_samples_per_second': 2.128, 'train_steps_per_second': 0.133, 'total_flos': 1.182847188221952e+16, 'train_loss': 1.8966534461975098, 'epoch': 0.38})